In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt

In [8]:
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

IMG_SIZE = 128;
NUM_CLASSES = 10;
batch_size = 16;
size = (IMG_SIZE,IMG_SIZE);

(ds_train, ds_test), ds_info = tfds.load(
    "imagenette", split=['train','validation'], with_info=True, as_supervised=True
)

ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

# ds_train = ds_train.map(lambda image, label: (tf.expand_dims(tf.image.resize(image, size),axis=0), label))
# ds_test = ds_test.map(lambda image, label: (tf.expand_dims(tf.image.resize(image, size),axis=0), label))


In [9]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = img_augmentation(inputs)
outputs = MobileNetV2(
    include_top=True,
    weights=None,
    alpha=0.5,
    classes=10,
    input_shape=(128,128,3),
    
)(x)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [10]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 img_augmentation (Sequentia  (None, 128, 128, 3)      0         
 l)                                                              
                                                                 
 mobilenetv2_0.50_128 (Funct  (None, 10)               719034    
 ional)                                                          
                                                                 
Total params: 719,034
Trainable params: 700,490
Non-trainable params: 18,544
_________________________________________________________________


In [11]:
epochs = 5  # @param {type: "slider", min:10, max:100}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/5


591/591 - 106s - loss: 2.1478 - accuracy: 0.2401 - val_loss: 2.3424 - val_accuracy: 0.0987 - 106s/epoch - 179ms/step
Epoch 2/5
591/591 - 77s - loss: 1.8749 - accuracy: 0.3495 - val_loss: 2.4595 - val_accuracy: 0.0987 - 77s/epoch - 130ms/step
Epoch 3/5
591/591 - 77s - loss: 1.7260 - accuracy: 0.4067 - val_loss: 2.6020 - val_accuracy: 0.0908 - 77s/epoch - 130ms/step
Epoch 4/5
591/591 - 78s - loss: 1.6164 - accuracy: 0.4524 - val_loss: 2.7180 - val_accuracy: 0.1069 - 78s/epoch - 131ms/step
Epoch 5/5
591/591 - 77s - loss: 1.5243 - accuracy: 0.4896 - val_loss: 2.7168 - val_accuracy: 0.1069 - 77s/epoch - 130ms/step


In [12]:
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/5
591/591 - 78s - loss: 1.4542 - accuracy: 0.5114 - val_loss: 2.7592 - val_accuracy: 0.0908 - 78s/epoch - 131ms/step
Epoch 2/5
591/591 - 77s - loss: 1.3965 - accuracy: 0.5334 - val_loss: 2.3410 - val_accuracy: 0.1515 - 77s/epoch - 131ms/step
Epoch 3/5
591/591 - 77s - loss: 1.3407 - accuracy: 0.5539 - val_loss: 1.7491 - val_accuracy: 0.4171 - 77s/epoch - 130ms/step
Epoch 4/5
591/591 - 77s - loss: 1.3011 - accuracy: 0.5640 - val_loss: 1.6871 - val_accuracy: 0.5298 - 77s/epoch - 131ms/step
Epoch 5/5
591/591 - 78s - loss: 1.2479 - accuracy: 0.5830 - val_loss: 3.5843 - val_accuracy: 0.3696 - 78s/epoch - 132ms/step


With batch size of 16, about 15% GPU utilization (~1TFLOP) and 77 seconds/epoch

takes 7 epochs to perform better than random guessing (10% accuracy)

takes 9 epochs to achieve better than 50% accuracy